In [3]:
#Import packages
import numpy as np
import scipy as sp
import matplotlib as ml
import os
import sys
flopypth = os.path.join('..', '..', 'flopy')
if flopypth not in sys.path:
    sys.path.append(flopypth)
import flopy
import flopy.utils.binaryfile as bf
import subprocess
import matplotlib.pyplot as plt
import shutil
from scipy.stats import uniform
import scipy.io as io
import datetime
from scipy.stats import uniform
from pyDOE import *

In [18]:
# Plot settings
plotContours = True
plotHydrograph = True
modflowSilent = False

# Delete modflow files after use?
deleteFiles = True

# Save output?
saveOutput = True

# Parameter sample inputs
hk_min = 1.e-3
hk_max = 1.
vka_min = 1.e-3
vka_max = 1.e-2
numWells = 2
pump_min = -50000.
pump_max = -3000.
sampleSize = 5
sy_min = .1
sy_max = .3

# Fixed Parameter Definitions
# Model domain and grid definition
Lx = 1000.
Ly = 1000.
ztop = 1000.
zbot = 0.
nlay = 1
nrow = 50
ncol = 50
delr = Lx / ncol
delc = Ly / nrow
delv = (ztop - zbot) / nlay
botm = np.linspace(ztop, zbot, nlay + 1)
sy = 2.5e-1
ss = 4.e-7
laytyp = 1  # 1 = unconfined, 0 = confined
hdry = 0    # dry cell head set to this number
mxiter = 300
hclose = 1e-1

# Variables for the BAS package
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)    # Not sure if GW uses 0 or 1
strt = 1000 * np.ones((nlay, nrow, ncol), dtype=np.float32)     # Starting head

# Time step parameters
nper = 1    # number of stress periods
perlen_max = 3000     # length of stress period
perlen_min = 10
nstp = 500      # Number of time steps per stress period
steady = [False]

# Well locations
wpt1 = (0, 20-1, 37-1)
wpt2 = (0, 37-1, 18-1)

# Output control
spd = {(0, 0): ['print head', 'save head']}

In [14]:
sampleSize
kwargs = {"hk": [.001, .01], "vka": [.0002, .002], 'pumpVol': [1000, 5000]}
numParam = len(kwargs)
lhd = lhs(numParam, samples=sampleSize)
params = {}
i = 0
for key, value in kwargs.items():
    loc = value[0]
    scale = value[1] - value[0]
    sample = uniform(loc=loc, scale=scale).ppf(lhd[:, i])
    params[key] = sample
    i += 1
params

{'hk': array([ 0.00172323,  0.00457344,  0.00890958,  0.00764122,  0.00609154]),
 'pumpVol': array([ 3735.0563533 ,  2687.83421493,  4835.15317162,  1269.43441553,
         2146.5892269 ]),
 'vka': array([ 0.00193207,  0.0006365 ,  0.00047292,  0.00109946,  0.00132117])}

In [19]:
def genParamSamples(sampleSize, **kwargs):
    # kwargs contains parameters for LHS sampling. Each parameter is an array containing two 
    # values: the min paramter value and the max parameter value
    
    # Generate LHS samples
    numParam = len(kwargs)
    lhd = lhs(numParam, samples=sampleSize)
    
    # Loop through parameters, use LHS values to generate parameter samples
    params = {}
    i = 0
    for key, value in kwargs.items():
        loc = value[0]
        scale = value[1] - value[0]
        sample =  uniform(loc=loc, scale=scale).ppf(lhd[:, i])
        params[key] = sample
        i += 1 

    # params is a dictionary where keys are parameter names and values are arrays of samples
    return params

In [21]:
samples = genParamSamples(sampleSize=sampleSize, hk=[hk_min, hk_max], vka=[vka_min, vka_max], sy=[sy_min, sy_max])
samples
if 'hk' in samples:
    print('yaaaass')

yaaaass


In [13]:
# Name model this run
model_name = 'model' + str(i)

# Get pumping rate same and make stress period dictionary
pumping_rate1 = samples['pump_rate_1'][i]
pumping_rate2 = samples['pump_rate_2'][i]
wel_sp1 = [0, 19, 36, pumping_rate1] # Remember to use zero-based layer, row, column indices!
wel_sp2 = [0, 36, 17, pumping_rate2]
stress_period_data = {0: [wel_sp1, wel_sp2]}

# Get hydraulic conductivty sample
hk = samples['hk'][i]
vka = samples['vka'][i]

# Get perlen sample
perlen = samples['time'][i]

# Flopy objects
mf = flopy.modflow.Modflow(model_name, exe_name='./mf2005dbl')
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc,
                               top=ztop, botm=botm[1:],
                               nper=nper, perlen=perlen, nstp=nstp, steady=steady)
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)
lpf = flopy.modflow.ModflowLpf(mf, hk=hk, vka=vka, sy=sy, ss=ss, laytyp=laytyp, hdry=hdry)
pcg = flopy.modflow.ModflowPcg(mf, mxiter=mxiter, hclose=hclose)  # This has stuff like iterations for solver
wel = flopy.modflow.ModflowWel(mf, stress_period_data=stress_period_data)
oc = flopy.modflow.ModflowOc(mf, stress_period_data=spd)

# Write the model input files
mf.write_input()

# Run the model
success, modflow_output = mf.run_model(silent=modflowSilent, pause=False, report=True)
modflow_success.append(success)

# Create MODFLOW output file if there was an error
if not success:
    file = open('modflow_output' + str(i) + '.txt', 'w')
    for n in modflow_output:
        file.write(n + '\n')
    raise Warning('MODFLOW did not terminate normally.')

Exception: The program ./mf2005dbl does not exist or is not executable.

In [15]:
import numpy as np
hk_min = 1.e-3
hk_max = 1.
hk_avg = np.mean([hk_max, hk_min])
hk_avg

0.50049999999999994

In [26]:
# Make row spacings, units in meters
delr = np.zeros(84)
delr[0:13] = 10.E3  # 10 km
delr[13:18] = 6.E3
delr[18:25] = 3.5E3
delr[25:28] = 1.8E3
delr[28:45] = 0.8E3
delr[45:68] = 1.E3
delr[68:77] = 3.5E3
delr[73] = 5.E3
delr[74] = 6.E3
delr[75:77] = 7.E3
delr[77:84] = 10.E3
xlength = sum(delr)

# Make column spacings, units in meters
delc = np.zeros(62)
delc[0:10] = 10.E3
delc[10:14] = 7.E3
delc[14:19] = 5.E3
delc[19:22] = 2.E3
delc[22:50] = 1.E3
delc[50] = 2.E3
delc[51] = 4.E3
delc[52] = 6.E3
delc[53] = 8.E3
delc[54:-1] = 10.E3
ylength = sum(delc)

# Other grid paratmers
ztop = 315.
zbot = 0.
nlay = 1
nrow = np.size(delr)
ncol = np.size(delc)
delv = (ztop - zbot) / nlay
botm = np.linspace(ztop, zbot, nlay + 1)
laytyp = 1  # 1 = unconfined, 0 = confined
hdry = 0    # dry cell head set to this number
mxiter = 300
hclose = 1e-1

# Time step parameters
perlen = 365
nper = 30  # number of stress periods
nstp = 365  # Number of time steps per stress period
steady = [False] * nper
pumpingOn = [1] * nper

i = 1
model_name = 'model' + str(i)
mf = flopy.modflow.Modflow(model_name, exe_name='./mf2005dbl')

In [36]:
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delc, delc=delr,
                                   top=ztop, botm=botm[1:],
                                   nper=nper, perlen=perlen, nstp=[nstp]*nper, steady=steady)

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfdis.ModflowDis'> <class 'flopy.modflow.mfdis.ModflowDis'>
replacing existing Package...


In [82]:
kwargs ={'hk':.012, 'vka': .002}
laytyp = 1  # 1 = unconfined, 0 = confined
hdry = 0  # dry cell head set to this number

# Default paramter inputs if none given
hk = 1.e-3
vka = hk / 10
sy = .25
ss = 4.e-7 * 10

for key, value in kwargs.items():
    if key == 'hk':
        hk = value
    elif key == 'vka':
        vka = value
    elif key == 'sy':
        sy = value
    elif key == 'ss':
        ss = value
        
lpf = flopy.modflow.ModflowLpf(mf, hk=hk, vka=vka, sy=sy, ss=ss, laytyp=laytyp, hdry=hdry)

****Warning -- two packages of the same type:  <class 'flopy.modflow.mflpf.ModflowLpf'> <class 'flopy.modflow.mflpf.ModflowLpf'>
replacing existing Package...


In [47]:
i = 2
samples = {'hk': [.1, .2, .3], 'vka': [.01, .02, .03]}
samplesThisRun = {k: v[i] for k, v in samples.items()}
samplesThisRun

{'hk': 0.3, 'vka': 0.03}

In [49]:
samplesThisRun['hk']

0.3

In [70]:
def getGridCell_fromLatLong(lat, long, gridOffset):
    from pyproj import Proj
    import numpy as np

    # Define projection for UTM coordinaates in zone 38 (near riyadh)
    p = Proj(proj='utm', zone=38)

    # Get UTM coordinates for lower left corner of grid
    [gridx, gridy] = p(22, 46)

    # Get UTM coorindates for input lat long
    [x, y] = p(lat, long)
    
    # Get distance between input point and grid corner in lat/long axes
    delx = x - gridx
    dely = y - gridy
    
    # Convert to distance along grid axes
    theta = np.arctan(dely / delx)
    alpha = gridOffset
    h = np.sqrt(delx**2 + dely**2)
    a = h * np.cos(theta - alpha)
    b = h * np.sin(theta - alpha)

    return a, b

In [73]:
lat = 24.7
long = 46.8
[x,y] = getGridCell_fromLatLong(lat, long, 22.3)
[x,y]

[-229545.77510311993, 42771.740581155354]

In [1]:
from flopy.utils.reference import SpatialReference
import inspect
inspect.getfile(SpatialReference.get_rc)

'/usr/local/lib/python3.6/site-packages/flopy/utils/reference.py'